In [1]:
! pip install langchain==0.2.2
! pip install langchain-community==0.2.3
! pip install langchain-openai==0.1.8

In [2]:
deployment = "gpt-4o"
model = "gpt-4o"

# 说明

使用 langchain 之后，这两个环境变量要重命名一下，否则会报错。

```bash
OPENAI_API_BASE -> AZURE_OPENAI_ENDPOINT
OPENAI_API_KEY -> AZURE_OPENAI_API_KEY
```

注意，是重命名，不能并存，否则会报错。

重命名后，不影响 OpenAI API 的使用，即重命名前后，都可以正常使用。

# 提示词模板

In [3]:
import os
from langchain import PromptTemplate, LLMChain

# from langchain_openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI

# from langchain.chat_models import ChatOpenAI # 直接访问 OpenAI 的 API 

prompt_template = """Make a good name for a product, the product is described as follow: {product}. Please only return the best one. """

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务


# llm = AzureChatOpenAI(azure_deployment=deployment) # OK
llm = AzureChatOpenAI(model=deployment,
                      temperature=0, max_tokens=200)

# deprecated
llm_chain = LLMChain(prompt=PromptTemplate.from_template(prompt_template), llm=llm)
result = llm_chain.invoke("colorful socks")
print(result)
# print("result: ", result.content, "token_usage: ", result.response_metadata.get("token_usage"))

products = [{"product": "'cloudnative devops platform'"},
            {"product": "'Noise cancellation headphone'"},
            {"product": "colorful socks"}]
result = llm_chain.apply(products)
print(result)

/Users/gevin/projects/pycharm/ai/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'product': 'colorful socks', 'text': 'VividSox'}
[{'text': 'CloudForge'}, {'text': 'SilentWave Pro'}, {'text': 'VividSoles'}]


In [4]:
prompt = PromptTemplate.from_template(prompt_template)
llm_chain = prompt | llm

result = llm_chain.invoke("colorful socks")
print("result: ", result.content, "token_usage: ", result.response_metadata.get("token_usage"))


result:  VividSoles token_usage:  {'completion_tokens': 4, 'prompt_tokens': 33, 'total_tokens': 37}


# API调用链
## HTTP request chain

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.llm_requests import LLMRequestsChain
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=deployment, temperature=0, max_tokens=200)


def query_baidu(question: str):
    # template = """
    # You are a search engine.
    # You are given a question, please search the internet and return the answer.
    # Question: {question}
    # Answer:
    # """
    template = """
      Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:
    """
    prompt = PromptTemplate(
        input_variables=["query", "requests_result"],
        template=template,
    )

    inputs = {
        "query": question,
        "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
    }

    # LLMChain deprecated
    chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt),
                             output_key="query_info", verbose=True)
    # llm_chain=llm | prompt
    # chain = LLMRequestsChain(llm_chain=llm_chain,
    #                           output_key="query_info", verbose=True)
    res = chain.invoke(inputs)
    return res


result = query_baidu("今天杭州天气")
print("result: ", result)



> Entering new LLMRequestsChain chain...

> Finished chain.
result:  {'query': '今天杭州天气', 'url': 'http://www.baidu.com/s?wd=今天杭州天气', 'query_info': 'Extracted: 今天杭州天气为多云，气温19~25°C，东风2级，空气质量优。'}


In [6]:
# import os
# os.environ["SERPER_API_KEY"] = ""
# https://serper.dev
from langchain.utilities import GoogleSerperAPIWrapper


def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)


query_web("今天杭州天气？")

'73°F'

# 调用链

In [7]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.sequential import SequentialChain 
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=deployment, temperature=0, max_tokens=200)

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""

summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key="summary")

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)

content = """
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
"""

res = overall_chain.invoke(content)

print("summary:"+res["summary"])

print("中文:"+res["translated"])



> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a framework for developing data-aware and agentic language model applications with modular components and off-the-shelf chains.
中文:LangChain 是一个用于开发数据感知和代理语言模型应用程序的框架，具有模块化组件和现成的链条。
